In [1]:
import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
def get_dataloaders(data_dir, batch_size=32):
    transform = transforms.Compose(
        [
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )

    train_dataset = datasets.ImageFolder(os.path.join(data_dir, "train"), transform)
    val_dataset = datasets.ImageFolder(os.path.join(data_dir, "val"), transform)
    test_dataset = datasets.ImageFolder(os.path.join(data_dir, "test"), transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader, len(train_dataset.classes)

In [3]:
data_dir = r"E:\Semester 7\Selected Topics in Artificial Intelligence-1\Project\Repo\Dangerous-Farm-Insects-Classification\data\processed\farm_insects\splits"
train_loader, val_loader, test_loader, num_classes = get_dataloaders(
    data_dir, batch_size=32
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [4]:
googlenet = models.googlenet(weights=models.GoogLeNet_Weights.IMAGENET1K_V1)
googlenet.fc = nn.Linear(1024, num_classes)
googlenet = googlenet.to(device)

Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to C:\Users\zeadm/.cache\torch\hub\checkpoints\googlenet-1378be20.pth


100%|██████████| 49.7M/49.7M [00:36<00:00, 1.44MB/s]


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(googlenet.parameters(), lr=1e-4)
num_epochs = 20

In [6]:
for epoch in range(num_epochs):
    googlenet.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = googlenet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}] - Training Loss: {avg_loss:.4f}")

c:\Python311\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [1/20] - Training Loss: 2.7208
Epoch [2/20] - Training Loss: 2.6383
Epoch [3/20] - Training Loss: 2.5534
Epoch [4/20] - Training Loss: 2.4681
Epoch [5/20] - Training Loss: 2.3847
Epoch [6/20] - Training Loss: 2.3156
Epoch [7/20] - Training Loss: 2.2439
Epoch [8/20] - Training Loss: 2.1626
Epoch [9/20] - Training Loss: 2.0863
Epoch [10/20] - Training Loss: 2.0217
Epoch [11/20] - Training Loss: 1.9536
Epoch [12/20] - Training Loss: 1.8695
Epoch [13/20] - Training Loss: 1.8088
Epoch [14/20] - Training Loss: 1.7484
Epoch [15/20] - Training Loss: 1.6674
Epoch [16/20] - Training Loss: 1.6083
Epoch [17/20] - Training Loss: 1.5395
Epoch [18/20] - Training Loss: 1.4866
Epoch [19/20] - Training Loss: 1.4165
Epoch [20/20] - Training Loss: 1.3749


In [ ]:
googlenet.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = googlenet(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average="weighted", zero_division=0)
recall = recall_score(all_labels, all_preds, average="weighted", zero_division=0)
f1 = f1_score(all_labels, all_preds, average="weighted", zero_division=0)

print("\nFinal Performance Metrics:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")


 Final Performance Metrics:
Accuracy : 0.6297
Precision: 0.6656
Recall   : 0.6297
F1 Score : 0.6223
